# Wavelet Scattering Transform

The wavelet scattering transform is a new technique for signal processing based on networks of wavelet convolutions.
It was pioneered by Stéphane Mallat in Paris in the early 2010s [(Mallat, 2012)](https://arxiv.org/abs/1101.2286).
The technique can be applied to 1D signals like time series, 2D signals like images, or higher dimensional signals. 
In the past decade, it has found many applications in physics, music, computer vision, and finance. 

Seminal references include:

- [Mallat (2012)](https://arxiv.org/abs/1101.2286) for signal processing and functional analysis,

- [Bruna & Mallat (2012)](https://arxiv.org/abs/1203.1513) for image recognition,

- [Anden & Mallat (2014)](https://arxiv.org/pdf/1304.6763) for audio classification,

- [Allys et al (2019)](https://arxiv.org/abs/1905.01372) for astrophysics,

- [Regaldo-Saint Blancard et al (2021)](https://arxiv.org/abs/2102.03160) for astrophysics,

- [Morel et al (2022)](https://arxiv.org/abs/2204.10177) for finance,

- [Cheng et al (2023)](https://arxiv.org/pdf/2306.17210) for physics.

We refer you to these references for technical details. Here we will focus on the intuition and in problem class you will implement a classifier using the scattering transform.
In the remainder of this lecture we will refer to the wavelet scattering transform as the *scattering transform*.
